# Installation 

In [63]:
import pandas as pd
import numpy as np
import requests
import time
import neo4j
import json
from neo4j import GraphDatabase
import re

In [64]:
API_KEY = '39Lmr8g-EvTQ_EuO1ZnoNtri4K5NXh9cBB4zUl6vpV0' 

# Merging Datasets

In [65]:
zhvi = pd.read_csv('zhvi_county.csv')
# print(zhvi)

In [66]:
variables_definations_for_B17017 = {
    "B17017_002E":  "Income in the past 12 months below poverty level",
    "B17017_031E": "Income in the past 12 months at or above poverty level"
}

index_for_variables_B17017 = {
    "B17017_002E": 4,
    "B17017_031E": 120,
    "GEO_ID" : 236,
    "NAME" : 237
}

index_for_variables_B17017_2010_2011 = {
    "B17017_002E": 3,
    "B17017_031E": 61,
    "GEO_ID" : 0,
    "NAME" : 119
}

In [173]:
B17017_data_files = ['B17017_Data_2010.json', 'B17017_Data_2011.json', 'B17017_Data_2012.json', 'B17017_Data_2013.json', 'B17017_Data_2014.json', 'B17017_Data_2015.json', 'B17017_Data_2016.json', 'B17017_Data_2017.json', 'B17017_Data_2018.json', 'B17017_Data_2019.json', 'B17017_Data_2021.json','B17017_Data_2022.json']
B17017_data = {}
for i in range(len(B17017_data_files)):
    B17017_json = json.load(open(B17017_data_files[i]))
    temp = []

    for var in B17017_json:
        if(B17017_data_files[i] == 'B17017_Data_2010.json' or B17017_data_files[i] == 'B17017_Data_2011.json'):
            temp.append([var[index_for_variables_B17017_2010_2011['GEO_ID']], 
                     var[index_for_variables_B17017_2010_2011['NAME']], 
                      var[index_for_variables_B17017_2010_2011['B17017_002E']],
                       var[index_for_variables_B17017_2010_2011['B17017_031E']]])
        else:
          temp.append([var[index_for_variables_B17017['GEO_ID']], 
                     var[index_for_variables_B17017['NAME']], 
                      var[index_for_variables_B17017['B17017_002E']],
                       var[index_for_variables_B17017['B17017_031E']]])
    
    B17017_data[re.search('Data_(.+?).json', B17017_data_files[i]).group(1)] = temp

In [174]:
B17017_data['2010'][0]

['GEO_ID', 'NAME', 'B17017_002E', 'B17017_031E']

In [175]:
B17017_data['2010'][1]

['0500000US39151', 'Stark County, Ohio', '18744', '130378']

In [88]:
variables_definations_for_S2502 = {
    "S2502_C01_001E":  "Total Occupied Housing Units",
    "S2502_C01_008E": "two or more races",
    "S2502_C01_009E" : "Hispanic or Latino origin",
    "S2502_C01_010E" : "White alone, not Hispanic or Latino",
    "S2502_C01_011E" : "under 35 years old",
    "S2502_C01_012E" : "35 to 44 years old",
    "S2502_C01_013E" : "45 to 54 years old",
    "S2502_C01_014E" : "55 to 64 years old",
    "S2502_C01_015E" : "65 to 74 years old",
    "S2502_C01_016E" : "75 to 84 years old",
    "S2502_C01_017E" : "85 years and over",
    "S2502_C01_018E" : "Less than high school graduate",
    "S2502_C01_019E" : "High school graduate (includes equivalency)",
    "S2502_C01_020E" : "Some college or associate's degree",
    "S2502_C01_021E" : "Bachelor's degree or higher",
}

index_for_variables_S2502 = {
    "S2502_C01_001E": 2,
    "S2502_C01_008E": 30,
    "S2502_C01_009E" : 34,
    "S2502_C01_010E" : 38,
    "S2502_C01_011E": 42,
    "S2502_C01_012E" : 46,
    "S2502_C01_013E": 50,
    "S2502_C01_014E" : 54,
    "S2502_C01_015E" : 58,
    "S2502_C01_016E" : 62,
    "S2502_C01_017E" : 66,
    "S2502_C01_018E" : 70,
    "S2502_C01_019E" : 74,
    "S2502_C01_020E" : 78,
    "S2502_C01_021E" : 82,
    "GEO_ID" : 0,
    "NAME" : 1
}

index_for_variables_S2502_2010_2011 = {
    "S2502_C01_001E": 1,
    "S2502_C01_008E": 15,
    "S2502_C01_009E" : 17,
    "S2502_C01_010E" : 19,
    "S2502_C01_011E": 21,
    "S2502_C01_012E" : 23,
    "S2502_C01_013E": 25,
    "S2502_C01_014E" : 27,
    "S2502_C01_015E" : 29,
    "S2502_C01_016E" : 31,
    "S2502_C01_017E" : 33,
    "S2502_C01_018E" : 35,
    "S2502_C01_019E" : 37,
    "S2502_C01_020E" : 39,
    "S2502_C01_021E" : 41,
    "GEO_ID" : 0,
    "NAME" : 169
}

In [151]:
S2502_data_files = ['S2502_Data_2010.json', 'S2502_Data_2011.json', 'S2502_Data_2012.json', 'S2502_Data_2013.json', 'S2502_Data_2014.json', 'S2502_Data_2015.json', 'S2502_Data_2016.json', 'S2502_Data_2017.json', 'S2502_Data_2018.json', 'S2502_Data_2019.json', 'S2502_Data_2021.json','S2502_Data_2022.json']
S2502_data = {}
for i in range(len(S2502_data_files)):
    S2502_json = json.load(open(S2502_data_files[i]))
    temp = []

    for var in S2502_json:
        if(S2502_data_files[i] == 'S2502_Data_2010.json' or S2502_data_files[i] == 'S2502_Data_2011.json'):
            if(var[0] != "GEO_ID"):
              temp.append([var[index_for_variables_S2502_2010_2011['GEO_ID']],
                      var[index_for_variables_S2502_2010_2011['NAME']],
                      var[index_for_variables_S2502_2010_2011['S2502_C01_001E']],
                          round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_008E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_008E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_008E']]) > 0 ) else 0,
                            round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_009E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_009E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_009E']]) > 0 ) else 0,
                              round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_010E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_010E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_010E']]) > 0 ) else 0,
                                round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_011E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) > 0 ) else 0,
                                  round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_012E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_012E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_012E']]) > 0 ) else 0,
                                    round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_013E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_013E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_013E']]) > 0 ) else 0,
                                      round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_014E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_014E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_014E']]) > 0 ) else 0,
                                        round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_015E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_015E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_015E']]) > 0 ) else 0,
                                          round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_016E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_016E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_016E']]) > 0 ) else 0,
                                            round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_017E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_017E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_017E']]) > 0 ) else 0,
                                              round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_018E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_018E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_018E']]) > 0 ) else 0,
                                                round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_019E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_019E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_019E']]) > 0 ) else 0,
                                                  round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_020E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_020E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_020E']]) > 0 ) else 0,
                                                    round(0.01 * float(var[index_for_variables_S2502_2010_2011['S2502_C01_021E']]) * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']])) if (var[index_for_variables_S2502_2010_2011['S2502_C01_021E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_021E']]) > 0 ) else 0,
                                                     round(float(100 - float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) if (var[index_for_variables_S2502_2010_2011['S2502_C01_011E']] != None and float(var[index_for_variables_S2502_2010_2011['S2502_C01_011E']]) > 0) else 0) * 0.01 * int(var[index_for_variables_S2502_2010_2011['S2502_C01_001E']]))])
            else:
                  temp.append([var[index_for_variables_S2502_2010_2011['GEO_ID']],
                      var[index_for_variables_S2502_2010_2011['NAME']],
                        var[index_for_variables_S2502_2010_2011['S2502_C01_001E']],
                          var[index_for_variables_S2502_2010_2011['S2502_C01_008E']],
                            var[index_for_variables_S2502_2010_2011['S2502_C01_009E']],
                              var[index_for_variables_S2502_2010_2011['S2502_C01_010E']],
                                var[index_for_variables_S2502_2010_2011['S2502_C01_011E']],
                                  var[index_for_variables_S2502_2010_2011['S2502_C01_012E']],
                                    var[index_for_variables_S2502_2010_2011['S2502_C01_013E']],
                                      var[index_for_variables_S2502_2010_2011['S2502_C01_014E']],
                                        var[index_for_variables_S2502_2010_2011['S2502_C01_015E']],
                                          var[index_for_variables_S2502_2010_2011['S2502_C01_016E']],
                                            var[index_for_variables_S2502_2010_2011['S2502_C01_017E']],
                                              var[index_for_variables_S2502_2010_2011['S2502_C01_018E']],
                                                var[index_for_variables_S2502_2010_2011['S2502_C01_019E']],
                                                  var[index_for_variables_S2502_2010_2011['S2502_C01_020E']],
                                                    var[index_for_variables_S2502_2010_2011['S2502_C01_021E']],
                                                      "Above 35 years old"])
                          
        elif(S2502_data_files[i] in ['S2502_Data_2012.json', 'S2502_Data_2013.json', 'S2502_Data_2014.json', 'S2502_Data_2015.json', 'S2502_Data_2016.json']):
            if (var[0] != "GEO_ID"): 
              temp.append([var[index_for_variables_S2502['GEO_ID']],
                     var[index_for_variables_S2502['NAME']],
                       var[index_for_variables_S2502['S2502_C01_001E']] if (var[index_for_variables_S2502['S2502_C01_001E']] != None) else 0,
                          round(0.01 * float(var[index_for_variables_S2502['S2502_C01_008E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_008E']] != None and float(var[index_for_variables_S2502['S2502_C01_008E']]) > 0) else 0,
                            round(0.01 * float(var[index_for_variables_S2502['S2502_C01_009E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_009E']] != None and float(var[index_for_variables_S2502['S2502_C01_009E']]) > 0) else 0,
                              round(0.01 * float(var[index_for_variables_S2502['S2502_C01_010E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_010E']] != None and float(var[index_for_variables_S2502['S2502_C01_010E']]) > 0) else 0,
                                round(0.01 * float(var[index_for_variables_S2502['S2502_C01_011E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_011E']] != None and float(var[index_for_variables_S2502['S2502_C01_011E']]) > 0) else 0,
                                  round(0.01 * float(var[index_for_variables_S2502['S2502_C01_012E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_012E']] != None) and float(var[index_for_variables_S2502['S2502_C01_012E']]) > 0 else 0, 
                                    round(0.01 * float(var[index_for_variables_S2502['S2502_C01_013E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_013E']] != None and float(var[index_for_variables_S2502['S2502_C01_013E']]) > 0) else 0,
                                      round(0.01 * float(var[index_for_variables_S2502['S2502_C01_014E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_014E']] != None and float(var[index_for_variables_S2502['S2502_C01_014E']]) > 0) else 0,
                                        round(0.01 * float(var[index_for_variables_S2502['S2502_C01_015E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_015E']] != None and float(var[index_for_variables_S2502['S2502_C01_015E']]) > 0) else 0,
                                          round(0.01 * float(var[index_for_variables_S2502['S2502_C01_016E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_016E']] != None and float(var[index_for_variables_S2502['S2502_C01_016E']]) > 0) else 0,
                                            round(0.01 * float(var[index_for_variables_S2502['S2502_C01_017E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_017E']] != None and float(var[index_for_variables_S2502['S2502_C01_017E']]) > 0) else 0,
                                              round(0.01 * float(var[index_for_variables_S2502['S2502_C01_018E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_018E']] != None and float(var[index_for_variables_S2502['S2502_C01_018E']]) > 0) else 0,
                                                round(0.01 * float(var[index_for_variables_S2502['S2502_C01_019E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_019E']] != None and float(var[index_for_variables_S2502['S2502_C01_019E']]) > 0) else 0,
                                                  round(0.01 * float(var[index_for_variables_S2502['S2502_C01_020E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_020E']] != None and float(var[index_for_variables_S2502['S2502_C01_020E']]) > 0) else 0,
                                                    round(0.01 * float(var[index_for_variables_S2502['S2502_C01_021E']]) * int(var[index_for_variables_S2502['S2502_C01_001E']])) if (var[index_for_variables_S2502['S2502_C01_021E']] != None and float(var[index_for_variables_S2502['S2502_C01_021E']]) > 0) else 0,
                                                      round(float(100 - float(var[index_for_variables_S2502['S2502_C01_011E']]) if (var[index_for_variables_S2502['S2502_C01_011E']] != None and float(var[index_for_variables_S2502['S2502_C01_011E']]) > 0) else 0) * 0.01 * int(var[index_for_variables_S2502['S2502_C01_001E']]))])
            else:
                  temp.append([var[index_for_variables_S2502['GEO_ID']],
                      var[index_for_variables_S2502['NAME']],
                        var[index_for_variables_S2502['S2502_C01_001E']],
                          var[index_for_variables_S2502['S2502_C01_008E']],
                            var[index_for_variables_S2502['S2502_C01_009E']],
                              var[index_for_variables_S2502['S2502_C01_010E']],
                                var[index_for_variables_S2502['S2502_C01_011E']],
                                  var[index_for_variables_S2502['S2502_C01_012E']],
                                    var[index_for_variables_S2502['S2502_C01_013E']],
                                      var[index_for_variables_S2502['S2502_C01_014E']],
                                        var[index_for_variables_S2502['S2502_C01_015E']],
                                          var[index_for_variables_S2502['S2502_C01_016E']],
                                            var[index_for_variables_S2502['S2502_C01_017E']],
                                              var[index_for_variables_S2502['S2502_C01_018E']],
                                                var[index_for_variables_S2502['S2502_C01_019E']],
                                                  var[index_for_variables_S2502['S2502_C01_020E']],
                                                    var[index_for_variables_S2502['S2502_C01_021E']],
                                                      "Above 35 years old"])
    
        else:
            temp.append([var[index_for_variables_S2502['GEO_ID']],
                     var[index_for_variables_S2502['NAME']],
                       var[index_for_variables_S2502['S2502_C01_001E']],
                         var[index_for_variables_S2502['S2502_C01_008E']],
                           var[index_for_variables_S2502['S2502_C01_009E']],
                             var[index_for_variables_S2502['S2502_C01_010E']],
                               var[index_for_variables_S2502['S2502_C01_011E']],
                                 var[index_for_variables_S2502['S2502_C01_012E']],
                                   var[index_for_variables_S2502['S2502_C01_013E']],
                                     var[index_for_variables_S2502['S2502_C01_014E']],
                                       var[index_for_variables_S2502['S2502_C01_015E']],
                                         var[index_for_variables_S2502['S2502_C01_016E']],
                                           var[index_for_variables_S2502['S2502_C01_017E']],
                                             var[index_for_variables_S2502['S2502_C01_018E']],
                                               var[index_for_variables_S2502['S2502_C01_019E']],
                                                 var[index_for_variables_S2502['S2502_C01_020E']],
                                                   var[index_for_variables_S2502['S2502_C01_021E']],
                                                    int(var[index_for_variables_S2502['S2502_C01_001E']]) - int(var[index_for_variables_S2502['S2502_C01_011E']] if (var[index_for_variables_S2502['S2502_C01_011E']] != None) else 0)  if (var[0] != "GEO_ID") else "Above 35 years old"] )
    
    S2502_data[re.search('Data_(.+?).json', S2502_data_files[i]).group(1)] = temp
    

In [14]:
variables_definations_for_S2503 = {
    "S2503_C01_026E" : "Less than $20,000,Less than 20 percent",
    "S2503_C01_028E" : "Less than $20,000,30 percent or more",
    "S2503_C01_030E" : "$20,000 to $34,999,Less than 20 percent",
    "S2503_C01_032E" : "$20,000 to $34,999,30 percent or more",
    "S2503_C01_034E" : "$35,000 to $49,999,Less than 20 percent",
    "S2503_C01_036E" : "$35,000 to $49,999,30 percent or more",
    "S2503_C01_038E" : "$50,000 to $74,999,Less than 20 percent",
    "S2503_C01_040E" : "$50,000 to $74,999,30 percent or more",
    "S2503_C01_042E" :  "$75,000 or more!!Less than 20 percent",
    "S2503_C01_044E" :  "$75,000 or more!!30 percent or more"
}

index_for_variables_S2503 = {
    "S2503_C01_026E" : 102,
    "S2503_C01_028E" : 110,
    "S2503_C01_030E" : 118,
    "S2503_C01_032E" : 126,
    "S2503_C01_034E" : 134,
    "S2503_C01_036E" : 142,
    "S2503_C01_038E" : 150,
    "S2503_C01_040E" : 158,
    "S2503_C01_042E" : 166,
    "S2503_C01_044E" : 174,
    "GEO_ID" : 0,
    "NAME" : 1
}

In [15]:
S2503_json = json.load(open('S2503_Data.json'))

In [16]:
i = 0
for var in S2503_json[0]:
    print(f'{i} : {var} : {S2503_json[1][i]}')
    i += 1 

0 : GEO_ID : 0500000US01003
1 : NAME : Baldwin County, Alabama
2 : S2503_C01_001E : 98854
3 : S2503_C01_001EA : None
4 : S2503_C01_001M : 3781
5 : S2503_C01_001MA : None
6 : S2503_C01_002E : 4555
7 : S2503_C01_002EA : None
8 : S2503_C01_002M : 1777
9 : S2503_C01_002MA : None
10 : S2503_C01_003E : 945
11 : S2503_C01_003EA : None
12 : S2503_C01_003M : 567
13 : S2503_C01_003MA : None
14 : S2503_C01_004E : 3760
15 : S2503_C01_004EA : None
16 : S2503_C01_004M : 1562
17 : S2503_C01_004MA : None
18 : S2503_C01_005E : 4962
19 : S2503_C01_005EA : None
20 : S2503_C01_005M : 1542
21 : S2503_C01_005MA : None
22 : S2503_C01_006E : 4615
23 : S2503_C01_006EA : None
24 : S2503_C01_006M : 1547
25 : S2503_C01_006MA : None
26 : S2503_C01_007E : 6643
27 : S2503_C01_007EA : None
28 : S2503_C01_007M : 1940
29 : S2503_C01_007MA : None
30 : S2503_C01_008E : 10949
31 : S2503_C01_008EA : None
32 : S2503_C01_008M : 2363
33 : S2503_C01_008MA : None
34 : S2503_C01_009E : 15916
35 : S2503_C01_009EA : None
36 : S250

In [17]:
S2503_Data = []
for var in S2503_json:
    S2503_Data.append([var[index_for_variables_S2503['GEO_ID']],
                        var[index_for_variables_S2503['NAME']],
                        var[index_for_variables_S2503['S2503_C01_026E']],
                        var[index_for_variables_S2503['S2503_C01_028E']],
                        var[index_for_variables_S2503['S2503_C01_030E']],
                        var[index_for_variables_S2503['S2503_C01_032E']],
                        var[index_for_variables_S2503['S2503_C01_034E']],
                        var[index_for_variables_S2503['S2503_C01_036E']],
                        var[index_for_variables_S2503['S2503_C01_038E']],
                        var[index_for_variables_S2503['S2503_C01_040E']],
                        var[index_for_variables_S2503['S2503_C01_042E']],
                        var[index_for_variables_S2503['S2503_C01_044E']],
                        int(var[index_for_variables_S2503['S2503_C01_026E']]) + int(var[index_for_variables_S2503['S2503_C01_030E']]) if (var[0] != 'GEO_ID') else "less than 35k income, less 20 percent",
                        int(var[index_for_variables_S2503['S2503_C01_028E']]) + int(var[index_for_variables_S2503['S2503_C01_032E']]) if (var[0] != 'GEO_ID') else "less than 35k income, 30 percent or more",
                        int(var[index_for_variables_S2503['S2503_C01_034E']]) + int(var[index_for_variables_S2503['S2503_C01_038E']]) if (var[0] != 'GEO_ID') else "35k to 75k income, 20 percent or less",
                        int(var[index_for_variables_S2503['S2503_C01_036E']]) + int(var[index_for_variables_S2503['S2503_C01_040E']]) if (var[0] != 'GEO_ID') else "35k to 75k income, 30 percent or more"])
                       

In [18]:
np.shape(S2503_Data)

(849, 16)

In [19]:
import pandas as pd

# Define the column names for the DataFrame
column_names = [
    'GEO_ID', 'NAME', 'Less than $20,000,Less than 20 percent',
    'Less than $20,000,30 percent or more',
    '$20,000 to $34,999,Less than 20 percent',
    '$20,000 to $34,999,30 percent or more',
    '$35,000 to $49,999,Less than 20 percent',
    '$35,000 to $49,999,30 percent or more',
    '$50,000 to $74,999,Less than 20 percent',
    '$50,000 to $74,999,30 percent or more',
    '$75,000 or more!!Less than 20 percent',
    '$75,000 or more!!30 percent or more', 
    'Less than 35k income, less 20 percent',
    'Less than 35k income, 30 percent or more',
    '35k to 75k income, 20 percent or less',
    '35k to 75k income, 30 percent or more'
]

# Create the DataFrame
df_S2503 = pd.DataFrame(S2503_Data[1:], columns=column_names)  # Skip header row

# Display the DataFrame
print(df_S2503)


             GEO_ID                                  NAME  \
0    0500000US01003               Baldwin County, Alabama   
1    0500000US01015               Calhoun County, Alabama   
2    0500000US01043               Cullman County, Alabama   
3    0500000US01049                DeKalb County, Alabama   
4    0500000US01051                Elmore County, Alabama   
..              ...                                   ...   
843  0500000US72113          Ponce Municipio, Puerto Rico   
844  0500000US72127       San Juan Municipio, Puerto Rico   
845  0500000US72135       Toa Alta Municipio, Puerto Rico   
846  0500000US72137       Toa Baja Municipio, Puerto Rico   
847  0500000US72139  Trujillo Alto Municipio, Puerto Rico   

    Less than $20,000,Less than 20 percent  \
0                                     1149   
1                                      558   
2                                      629   
3                                     1019   
4                                    

# Populating the Zillow data

In [20]:
us_states = [
    ["AL", "Alabama"],
    ["AK", "Alaska"],
    ["AZ", "Arizona"],
    ["AR", "Arkansas"],
    ["CA", "California"],
    ["CO", "Colorado"],
    ["CT", "Connecticut"],
    ["DE", "Delaware"],
    ["FL", "Florida"],
    ["GA", "Georgia"],
    ["HI", "Hawaii"],
    ["ID", "Idaho"],
    ["IL", "Illinois"],
    ["IN", "Indiana"],
    ["IA", "Iowa"],
    ["KS", "Kansas"],
    ["KY", "Kentucky"],
    ["LA", "Louisiana"],
    ["ME", "Maine"],
    ["MD", "Maryland"],
    ["MA", "Massachusetts"],
    ["MI", "Michigan"],
    ["MN", "Minnesota"],
    ["MS", "Mississippi"],
    ["MO", "Missouri"],
    ["MT", "Montana"],
    ["NE", "Nebraska"],
    ["NV", "Nevada"],
    ["NH", "New Hampshire"],
    ["NJ", "New Jersey"],
    ["NM", "New Mexico"],
    ["NY", "New York"],
    ["NC", "North Carolina"],
    ["ND", "North Dakota"],
    ["OH", "Ohio"],
    ["OK", "Oklahoma"],
    ["OR", "Oregon"],
    ["PA", "Pennsylvania"],
    ["RI", "Rhode Island"],
    ["SC", "South Carolina"],
    ["SD", "South Dakota"],
    ["TN", "Tennessee"],
    ["TX", "Texas"],
    ["UT", "Utah"],
    ["VT", "Vermont"],
    ["VA", "Virginia"],
    ["WA", "Washington"],
    ["WV", "West Virginia"],
    ["WI", "Wisconsin"],
    ["WY", "Wyoming"]
]

def replace_state_abbreviation(state):
    for abbreviation, full_name in us_states:
        if state == abbreviation:
            return full_name
    return state

In [24]:
for state_abbr in zhvi['StateName'].unique():  
    state_name = replace_state_abbreviation(state_abbr)
    state_filtered_data = zhvi[zhvi['StateName'] == state_abbr]
    county_names = state_filtered_data['RegionName'].unique()
    print(state_name)
    print(len(county_names))
    print(county_names)


California
58
['Los Angeles County' 'San Diego County' 'Orange County'
 'Riverside County' 'San Bernardino County' 'Santa Clara County'
 'Alameda County' 'Sacramento County' 'Contra Costa County'
 'Fresno County' 'Kern County' 'San Francisco County' 'Ventura County'
 'San Joaquin County' 'San Mateo County' 'Stanislaus County'
 'Sonoma County' 'Tulare County' 'Solano County' 'Santa Barbara County'
 'Monterey County' 'Placer County' 'San Luis Obispo County'
 'Merced County' 'Santa Cruz County' 'Marin County' 'Butte County'
 'Yolo County' 'El Dorado County' 'Shasta County' 'Imperial County'
 'Madera County' 'Kings County' 'Napa County' 'Humboldt County'
 'Nevada County' 'Sutter County' 'Mendocino County' 'Yuba County'
 'Lake County' 'Tehama County' 'San Benito County' 'Tuolumne County'
 'Calaveras County' 'Siskiyou County' 'Amador County' 'Lassen County'
 'Glenn County' 'Del Norte County' 'Colusa County' 'Plumas County'
 'Inyo County' 'Mariposa County' 'Trinity County' 'Mono County'
 'Mod

In [ ]:
import sys
from neo4j.debug import watch


#watch("neo4j", out=sys.stdout) #Output debug to stdout 
#watch("neo4j", out=open('debugLogs.txt', 'w')) #Output debug to logfile 

URI = "bolt://host.docker.internal:7687"
AUTH = ("neo4j", "13781378")
def main():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            # Populate Neo4j with county data from original dataset
            for state_abbr in zhvi['StateName'].unique():   
                state_name = replace_state_abbreviation(state_abbr)    
                add_counties_for_state(session, state_name, state_abbr)
                print(f"Counties added for State {state_name}")

            # # Populate Neo4j with poverty data
            # populate_poverty_data(driver, B17017_data)
            # check_poverty_data(session)

            # # Populate Neo4j with housing data from S2502 dataset
            # populate_housing_data(driver, df_S2502)
            # check_housing_data(session, "S2502")

            # # Populate Neo4j with housing data from S2503 dataset
            # populate_housing_data_S2503(driver, df_S2503)
            # check_housing_data(session, "S2503")


# def check_poverty_data(session):
#     result = session.run("MATCH (c:County) WHERE c.IncomeBelowPoverty IS NOT NULL AND c.IncomeAtOrAbovePoverty IS NOT NULL RETURN COUNT(c) AS count")
#     count = result.single()["count"]
#     if count > 0:
#         print(f"Poverty data populated for {count} counties.")
#     else:
#         print("No poverty data populated.")



# def populate_housing_data_S2503(driver, df_S2503):
#     with driver.session(database="neo4j") as session:
#         for index, row in df_S2503.iterrows():
#             geo_id, name, *housing_data = row

#             # Extract county name and state abbreviation
#             county_info = name.split(", ")
#             county_name = county_info[0]
#             state_abbr = county_info[1]

#             # Match county with corresponding node in Neo4j
#             result = session.run("""
#                 MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
#                 RETURN c
#                 """, state_abbr=state_abbr, county_name=county_name)

#             for record in result:
#                 county_node = record["c"]

#                 # Update or create housing data properties
#                 housing_data_dict = {column: value for column, value in zip(df_S2503.columns[2:], housing_data)}
#                 county_node.update(housing_data_dict)
#                 print(f"Housing data (S2503) populated for County {county_name}.")


def add_counties_for_state(session, state_name, state_abbr):
    # Fetch county names for the given state
    state_filtered_data = zhvi[zhvi['StateName'] == state_abbr]
    county_names = state_filtered_data['RegionName'].unique()

    # Create or update state node
    session.write_transaction(create_or_update_state_node, state_name)

    # Create "HAS_COUNTY" relationships for the state
    for county_name in county_names:
        session.write_transaction(create_or_update_county_node, county_name)
        session.write_transaction(create_has_county_relationship, state_name, county_name)


def create_or_update_state_county_nodes(tx, state_name, county_names):
    # Create or update State node with given name
    tx.run("""
        CREATE (s:State {name: $name})
        """, name=state_name
    )


def create_or_update_county_node(tx, county_name):
    # Create or update County node with given name
    tx.run("""
       CREATE (c:County {name: $name})
        """, name=county_name
    )


def create_has_county_relationship(tx, state_name, county_name):
    # Create a "HAS_COUNTY" relationship between state and county
    tx.run("""
        MATCH (s:State {name: $state_name}), (c:County {name: $county_name})
        MERGE (s)-[:HAS_COUNTY]->(c)
        """, state_name=state_name, county_name=county_name
    )


# def populate_poverty_data(driver, B17017_data):
#     with driver.session(database="neo4j") as session:
#         for row in B17017_data[1:]:  # Skip header row
#             geo_id, name, below_poverty, at_or_above_poverty = row
#             county_info = name.split(", ")
#             county_name = county_info[0]
#             state_abbr = county_info[1]

#             # Match county with corresponding node in Neo4j
#             result = session.run("""
#                 MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
#                 RETURN c
#                 """, state_abbr=state_abbr, county_name=county_name)

#             for record in result:
#                 county_node = record["c"]

#                 # Update or create poverty level properties
#                 county_node.update({
#                     "IncomeBelowPoverty": below_poverty,
#                     "IncomeAtOrAbovePoverty": at_or_above_poverty
#                 })
#                 print(f"Poverty data populated for County {county_name}.")


# def populate_housing_data(driver, df_S2502):
#     with driver.session(database="neo4j") as session:
#         for index, row in df_S2502.iterrows():
#             geo_id, name, *housing_data, above_35 = row

#             # Extract county name and state abbreviation
#             county_info = name.split(", ")
#             county_name = county_info[0]
#             state_abbr = county_info[1]

#             # Match county with corresponding node in Neo4j
#             result = session.run("""
#                 MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
#                 RETURN c
#                 """, state_abbr=state_abbr, county_name=county_name)

#             for record in result:
#                 county_node = record["c"]

#                 # Update or create housing data properties
#                 housing_data_dict = {column: value for column, value in zip(df_S2502.columns[2:], housing_data)}
#                 housing_data_dict.update({"Above35YearsOld": above_35})

#                 county_node.update(housing_data_dict)
#                 print(f"Housing data (S2502) populated for County {county_name}.")


# def populate_housing_data_S2503(driver, df_S2503):
#     with driver.session(database="neo4j") as session:
#         for index, row in df_S2503.iterrows():
#             geo_id, name, *housing_data = row

#             # Extract county name and state abbreviation
#             county_info = name.split(", ")
#             county_name = county_info[0]
#             state_abbr = county_info[1]

#             # Match county with corresponding node in Neo4j
#             result = session.run("""
#                 MATCH (s:State {abbr: $state_abbr})-[:HAS_COUNTY]->(c:County {name: $county_name})
#                 RETURN c
#                 """, state_abbr=state_abbr, county_name=county_name)

#             for record in result:
#                 county_node = record["c"]

#                 # Update or create housing data properties
#                 housing_data_dict = {column: value for column, value in zip(df_S2503.columns[2:], housing_data)}
#                 county_node.update(housing_data_dict)
#                 print(f"Housing data (S2503) populated for County {county_name}.")



main()

# Populating the census data

In [ ]:
def populate_poverty_data(driver, df):
    with driver.session(database="neo4j") as session:
        for index, row in df.iterrows():
            geo_id, name, attribute1, attribute2 = row
            # Extract county name
            county_name = name.split(", ")[0]

            # Handle None values for attributes
            attribute1 = int(attribute1) if attribute1 is not None else 0
            attribute2 = int(attribute2) if attribute2 is not None else 0

            # Construct the Cypher query to update data for the county node
            query = """
                MERGE (c:County {name: $county_name})
                SET c.`Income in the past 12 months below poverty level` = $attribute1,
                    c.`Income in the past 12 months at or above poverty level` = $attribute2
            """
            # Execute the query with the data
            session.run(query, {
                "county_name": county_name,
                "attribute1": attribute1,
                "attribute2": attribute2
            })


def populate_housing_data_S2502(driver, df):
    with driver.session(database="neo4j") as session:
        for index, row in df.iterrows():
            geo_id, name, *housing_data = row

            # Extract county name
            county_name = name.split(", ")[0]

            # Check if housing data is missing
            if len(housing_data) != 16:
                print(f"Missing housing data for {county_name}.")
                continue

            # Construct the Cypher query to update housing data for the county node
            query = """
                MATCH (c:County {name: $county_name})
                SET c.`Total Occupied Housing Units` = $total_units,
                    c.`Two or More Races` = $two_or_more_races,
                    c.`Hispanic or Latino Origin` = $hispanic_or_latino,
                    c.`White Alone, Not Hispanic or Latino` = $white_alone_not_hispanic,
                    c.`Under 35 Years Old` = $under_35,
                    c.`35 to 44 Years Old` = $age_35_to_44,
                    c.`45 to 54 Years Old` = $age_45_to_54,
                    c.`55 to 64 Years Old` = $age_55_to_64,
                    c.`65 to 74 Years Old` = $age_65_to_74,
                    c.`75 to 84 Years Old` = $age_75_to_84,
                    c.`85 Years Old and Over` = $age_85_and_over,
                    c.`Median Value` = $median_value,
                    c.`Median Gross Rent` = $median_gross_rent,
                    c.`Median Selected Monthly Owner Costs with Mortgage` = $median_selected_monthly_owner_costs_with_mortgage,
                    c.`Median Selected Monthly Owner Costs without Mortgage` = $median_selected_monthly_owner_costs_without_mortgage,
                    c.`Median Rent` = $median_rent
            """
            # Execute the query with housing data
            session.run(query, {
                "county_name": county_name,
                "total_units": housing_data[0],
                "two_or_more_races": housing_data[1],
                "hispanic_or_latino": housing_data[2],
                "white_alone_not_hispanic": housing_data[3],
                "under_35": housing_data[4],
                "age_35_to_44": housing_data[5],
                "age_45_to_54": housing_data[6],
                "age_55_to_64": housing_data[7],
                "age_65_to_74": housing_data[8],
                "age_75_to_84": housing_data[9],
                "age_85_and_over": housing_data[10],
                "median_value": housing_data[11],
                "median_gross_rent": housing_data[12],
                "median_selected_monthly_owner_costs_with_mortgage": housing_data[13],
                "median_selected_monthly_owner_costs_without_mortgage": housing_data[14],
                "median_rent": housing_data[15]
            })


def populate_housing_data_S2503(driver, df):
    with driver.session(database="neo4j") as session:
        for index, row in df.iterrows():
            geo_id, name, *housing_data = row
            # Extract county name
            county_name = name.split(", ")[0]

            # Check if housing data is missing
            if len(housing_data) != 14:
                print(f"Missing housing data for {county_name}.")
                continue

            # Construct the Cypher query to update housing data for the county node
            query = """
                MATCH (c:County {name: $county_name})
                SET c.`Less than $20,000, Less than 20 percent` = $less_than_20000_less_than_20,
                    c.`Less than $20,000, 30 percent or more` = $less_than_20000_30_or_more,
                    c.`$20,000 to $34,999, Less than 20 percent` = $20000_to_34999_less_than_20,
                    c.`$20,000 to $34,999, 30 percent or more` = $20000_to_34999_30_or_more,
                    c.`$35,000 to $49,999, Less than 20 percent` = $35000_to_49999_less_than_20,
                    c.`$35,000 to $49,999, 30 percent or more` = $35000_to_49999_30_or_more,
                    c.`$50,000 to $74,999, Less than 20 percent` = $50000_to_74999_less_than_20,
                    c.`$50,000 to $74,999, 30 percent or more` = $50000_to_74999_30_or_more,
                    c.`$75,000 or more, Less than 20 percent` = $75000_or_more_less_than_20,
                    c.`$75,000 or more, 30 percent or more` = $75000_or_more_30_or_more,
                    c.`Less than 35k income, less 20 percent` = $less_than_35k_income_less_than_20,
                    c.`Less than 35k income, 30 percent or more` = $less_than_35k_income_30_or_more,
                    c.`35k to 75k income, 20 percent or less` = $35k_to_75k_income_20_or_less,
                    c.`35k to 75k income, 30 percent or more` = $35k_to_75k_income_30_or_more
            """
            # Execute the query with housing data
            session.run(query, {
                "county_name": county_name,
                "less_than_20000_less_than_20": housing_data[0],
                "less_than_20000_30_or_more": housing_data[1],
                "20000_to_34999_less_than_20": housing_data[2],
                "20000_to_34999_30_or_more": housing_data[3],
                "35000_to_49999_less_than_20": housing_data[4],
                "35000_to_49999_30_or_more": housing_data[5],
                "50000_to_74999_less_than_20": housing_data[6],
                "50000_to_74999_30_or_more": housing_data[7],
                "75000_or_more_less_than_20": housing_data[8],
                "75000_or_more_30_or_more": housing_data[9],
                "less_than_35k_income_less_than_20": housing_data[10],
                "less_than_35k_income_30_or_more": housing_data[11],
                "35k_to_75k_income_20_or_less": housing_data[12],
                "35k_to_75k_income_30_or_more": housing_data[13]
            })


URI = "http://localhost:7474/"
AUTH = ("neo4j", "13781378")
# Adjust the main function accordingly
def main():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            # Populate Neo4j with poverty data
            populate_poverty_data(driver, df_B17017)

            # Populate Neo4j with housing data from S2502 dataset
            populate_housing_data_S2502(driver, df_S2502)

            # Populate Neo4j with housing data from S2503 dataset
            populate_housing_data_S2503(driver, df_S2503)


main()


# Query to check if the Neo4j population works

In [ ]:
from neo4j import GraphDatabase

# Define the Neo4j connection details
URI = "neo4j+s://5dd34660.databases.neo4j.io"
USER = "neo4j"
PASSWORD = "a1yUVTX6sa3eGz7MSVAqED0zzIB_VUq4YAsSPlVB-JU"

# Function to execute the Cypher query and return the count
def execute_query_and_return_count(query):
    with GraphDatabase.driver(URI, auth=(USER, PASSWORD)) as driver:
        with driver.session() as session:
            result = session.run(query)
            count = result.single()["count"]
            return count

# Query to check if poverty data was populated
poverty_query = """
MATCH (c:County)
WHERE c.`Income in the past 12 months at or above poverty level` IS NOT NULL AND c.`Income in the past 12 months below poverty level` IS NOT NULL
RETURN COUNT(c) AS count
"""

# Query to check if housing data from S2502 dataset was populated
housing_s2502_query = """
MATCH (c:County)
WHERE c.`Total Occupied Housing Units` IS NOT NULL
RETURN COUNT(c) AS count
"""

# Query to check if housing data from S2503 dataset was populated
housing_s2503_query = """
MATCH (c:County)
WHERE c.`Less than $20,000, Less than 20 percent` IS NOT NULL
RETURN COUNT(c) AS count
"""

# Execute queries and print results
print("Poverty data populated for", execute_query_and_return_count(poverty_query), "counties.")
print("Housing data (S2502) populated for", execute_query_and_return_count(housing_s2502_query), "counties.")
print("Housing data (S2503) populated for", execute_query_and_return_count(housing_s2503_query), "counties.")
